## SQL Examples with Baseball

Here we show SQL constructs using the Lahman baseball dataset (downloaded from https://github.com/jknecht/baseball-archive-sqlite). We also show how to use a SQL database inside a Jupyter notebook.

First, we create a connection to the database. In this case, we are using a `SQLite` database. A good system to prototype database designs. To make the most of a database system, one would use some of the more powerful products: Oracle, Microsoft SQLServer, MySQL (MariaDB), PostgreSQL or other. In all cases, the way to create a connection to the server from Rmarkdown is the same.

In [6]:
import pandas as pd
import sqlite3

In [7]:
con = sqlite3.connect(r'data/lahman2016.sqlite')

### Select-From-Where

First, we write a query to get batting statistics for Washington Nationals in 2010: note the table *rename*. 

In [8]:
query = """
SELECT b.playerId, b.yearId, b.H, b.AB
FROM BATTING AS b
WHERE teamID = 'WAS' AND yearID = 2010
"""
df = pd.read_sql_query(query, con)
df.head(10)

,playerID,yearID,H,AB
0,atilalu01,2010,1,25
1,balesco01,2010,0,0
2,batismi01,2010,1,8
3,bergmja01,2010,0,0
4,bernaro01,2010,102,414
5,bisenjo01,2010,0,0
6,brunebr01,2010,0,0
7,burkeja02,2010,0,0
8,burnese01,2010,0,0
9,cappsma01,2010,0,1


### Expressions

The **select** clause can contain expressions (this is paralleled by the `assign` operation we saw previously)

In [9]:
query = """
SELECT b.playerId, b.yearId, b.AB, 1.0 * b.H / b.AB AS BP
FROM BATTING AS b
"""
df = pd.read_sql_query(query, con)
df.head()

,playerID,yearID,AB,BP
0,aardsda01,2004,0,NaN
1,aardsda01,2006,2,0.0
2,aardsda01,2007,0,NaN
3,aardsda01,2008,1,0.0
4,aardsda01,2009,0,NaN


### WHERE predicates

The **where** clause support a large number of different predicates and combinations thereof (this is parallel to the `query` operation)

In [10]:
query = """
SELECT b.playerId, b.yearID, b. teamId, b.AB, 1.0 * b.H / b.AB AS BP
FROM BATTING AS b
WHERE b.AB > 0 AND
  b.yearID > 2000 AND
  b.yearID < 2010 AND 
  b.teamID LIKE 'NY%'
"""
df = pd.read_sql_query(query, con)
df.head()

,playerID,yearID,teamID,AB,BP
0,abreubo01,2006,NYA,209,0.330144
1,abreubo01,2007,NYA,605,0.282645
2,abreubo01,2008,NYA,609,0.295567
3,aceveal01,2009,NYA,2,0.000000
4,agbaybe01,2001,NYN,296,0.277027


### ORDERING

We can include ordering (parallel to `sort_values`)

In [12]:
query = """
SELECT b.playerId, b.yearID, b. teamId, b.AB, 1.0 * b.H / b.AB AS BP
FROM BATTING AS b
WHERE b.AB > 0 AND
  b.yearID > 2000 AND
  b.yearID < 2010 AND 
  b.teamID LIKE 'NY%'
ORDER BY b.AB DESC, BP DESC
"""
df = pd.read_sql_query(query, con)
df.head()

,playerID,yearID,teamID,AB,BP
0,soriaal01,2002,NYA,696,0.300287
1,reyesjo01,2005,NYN,696,0.272989
2,reyesjo01,2008,NYN,688,0.296512
3,soriaal01,2003,NYA,682,0.290323
4,reyesjo01,2007,NYN,681,0.280470


### Group_by and Summarize

- What it does: Partition the tuples by the group attributes (*teamId* and *yearId* in this case), and do something (*compute avg* in this case) for each group
- Number of resulting tuples == Number of groups

In [13]:
query = """
SELECT b.teamId, b.yearId, avg(1.0 * b.H / b.AB) AS AVE_BP
FROM BATTING AS b
WHERE b.AB > 0 AND
  b.yearID > 2000 AND
  b.yearID < 2010
GROUP BY b.teamId, b.yearId
ORDER BY AVE_BP DESC
"""
df = pd.read_sql_query(query, con)
df.head()

,teamID,yearID,AVE_BP
0,TBA,2007,0.289207
1,MIN,2008,0.287575
2,SEA,2007,0.284548
3,SLN,2003,0.283298
4,MIN,2001,0.279198


For reference, this is how we would do this using 
the `pandas` operations we learned about previously.

```python
Batting
    .assign(BP = 1.0 * Batting.H / Batting.AB)
    .query('AB > 0 & ...')
    .groupby(['teamId', 'yearId'])
    .agg({'AB': ['mean']})
```

### Subqueries

Sometimes it's easier to nest queries like the one above into query and subquery

In [14]:
query = """
SELECT teamID, yearID, avg(BP) AS AVG_BP
FROM (SELECT b.teamId, b.yearId, 1.0 * b.H / b.AB AS BP
      FROM BATTING AS b
      WHERE b.AB > 0 AND
        b.yearID > 2000 AND
        b.yearID < 2010)
GROUP BY teamID, yearID
ORDER BY AVG_BP DESC;
"""

df = pd.read_sql_query(query, con)
df.head()

,teamId,yearId,AVG_BP
0,TBA,2007,0.289207
1,MIN,2008,0.287575
2,SEA,2007,0.284548
3,SLN,2003,0.283298
4,MIN,2001,0.279198


## Joins

List all players from California, playing in 2015

In [15]:
query = """
SELECT b.playerId, b.teamId, m.birthState
FROM Batting as b JOIN master as m on b.playerId == m.playerId
WHERE yearId = "2015" and m.birthState = "CA"
""" 

df = pd.read_sql_query(query, con)
df.head()

,playerID,teamID,birthState
0,alexasc01,KCA,CA
1,anderbr05,OAK,CA
2,anderco01,CLE,CA
3,andrima01,TBA,CA
4,arenano01,COL,CA


Finally, we close the connection to the database:

In [16]:
con.close()